##Bangla

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import re
import os
import random
import time
import warnings
from collections import defaultdict, Counter
import itertools
import math

warnings.filterwarnings("ignore")

# ---------------------------
# CONFIG
# ---------------------------
author = "bangla_pmi_graph_analysis.xlsx"
OUTPUT_DIR = "bangla_outputs_PMI"
FILE_NAME = f"{author}_PMI.xlsx"
# OUTPUT_EXCEL = os.path.join(OUTPUT_DIR, "bangla_pmi_graph_analysis.xlsx")
OUTPUT_EXCEL = os.path.join(OUTPUT_DIR, FILE_NAME)

# Thresholds for switching to approximate computations
MAX_NODES_EXACT = 500           # <= this: do exact heavy computations
BETWEENNESS_SAMPLE_K = 100      # 'k' parameter for approx betweenness
PATHS_SAMPLE_NODES = 100        # number of source nodes to sample for path-based metrics
RANDOM_SEED = 42

# PMI Parameters
WINDOW_SIZE = 4                 # sliding window size for co-occurrence
MIN_PMI = 0.0                   # minimum PMI threshold for edge creation

# create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print("PMI Co-occurrence Graph Analysis System Initialized\n")

# ---------------------------
# Text Preprocessing Functions
# ---------------------------
def clean_bengali_text(text):
    """Remove unnecessary punctuation, keep only Bengali characters and spaces"""
    # Remove punctuation but keep Bengali characters and spaces
    text = re.sub(r'[।,?!.;:"''""\(\)\[\]—–\-…]', '', text)
    # Keep only Bengali Unicode range and spaces
    text = re.sub(r'[^\u0980-\u09FF\s]', '', text)
    return text.strip()

def simple_tokenize(text):
    """Simple tokenization by splitting on whitespace"""
    return text.split()

def remove_stopwords(tokens):
    """Remove common Bengali stopwords"""
    stopwords = [
        'এবং', 'ও', 'একটি', 'এর', 'এই', 'একজন', '।', 'তথা',
        'সে', 'তার', 'তিনি', 'যে', 'কে', 'কি', 'হয়', 'হয়ে',
        'করে', 'করা', 'হল', 'হলো', 'আর', 'বা', 'না', 'নি'
    ]
    return [word for word in tokens if word not in stopwords and len(word) > 1]

def preprocess_text(text):
    """Complete preprocessing pipeline"""
    cleaned = clean_bengali_text(text)
    tokens = simple_tokenize(cleaned)
    tokens = remove_stopwords(tokens)
    return tokens

# ---------------------------
# PMI Co-occurrence Functions
# ---------------------------
def calculate_pmi_edges(tokens, window_size=WINDOW_SIZE, min_pmi=MIN_PMI):
    """Calculate PMI-based edges from tokens"""
    if len(tokens) < 2:
        return {}

    word_freq = Counter()
    pair_freq = Counter()
    total_windows = 0

    # Count word frequencies and co-occurrences within sliding windows
    for i in range(len(tokens) - window_size + 1):
        window = tokens[i:i + window_size]
        total_windows += 1

        # Count individual words
        for word in window:
            word_freq[word] += 1

        # Count word pairs
        for w1, w2 in itertools.combinations(window, 2):
            pair = tuple(sorted((w1, w2)))
            pair_freq[pair] += 1

    # Calculate PMI scores and create edges
    edges = {}
    for (w1, w2), c in pair_freq.items():
        if total_windows == 0:
            continue

        p_xy = c / total_windows
        p_x = word_freq[w1] / (total_windows * window_size)
        p_y = word_freq[w2] / (total_windows * window_size)

        if p_x > 0 and p_y > 0:
            pmi = math.log(p_xy / (p_x * p_y))
            if pmi > min_pmi:
                edges[(w1, w2)] = pmi

    return edges

def create_pmi_graph(tokens):
    """Create NetworkX graph from tokens using PMI co-occurrence"""
    edges = calculate_pmi_edges(tokens)

    G = nx.Graph()

    # Add all tokens as nodes
    for token in set(tokens):
        G.add_node(token)

    # Add edges based on PMI scores
    for (w1, w2), pmi_score in edges.items():
        G.add_edge(w1, w2, weight=pmi_score)

    return G

# ---------------------------
# Utilities for Approximations
# ---------------------------
def sample_nodes_list(G, k):
    n = G.number_of_nodes()
    nodes = list(G.nodes())
    if n <= k:
        return nodes
    return random.sample(nodes, k)

def approx_average_shortest_path(H, sample_nodes):
    lengths_accum = []
    count = 0
    for s in sample_nodes:
        try:
            lens = nx.single_source_dijkstra_path_length(H, s, weight="distance")
            lengths_accum.extend(list(lens.values()))
            count += len(lens)
        except Exception:
            continue
    if count == 0:
        return -1
    return float(sum(lengths_accum) / count)

def approx_wiener_index(H, sample_nodes):
    wsum = 0.0
    pairs_count = 0
    for s in sample_nodes:
        try:
            lens = nx.single_source_dijkstra_path_length(H, s, weight="distance")
        except Exception:
            continue
        for t, d in lens.items():
            if s < t:
                wsum += d
                pairs_count += 1
    return float(wsum) if pairs_count > 0 else -1

def approx_eccentricity(H, sample_nodes):
    ecc = {}
    for s in sample_nodes:
        try:
            lens = nx.single_source_dijkstra_path_length(H, s, weight="distance")
            if len(lens) == 0:
                ecc[s] = -1
            else:
                ecc[s] = max(lens.values())
        except Exception:
            ecc[s] = -1
    return ecc

# ---------------------------
# Comprehensive Graph Metrics
# ---------------------------
def compute_graph_metrics(G, H, name, max_nodes_exact=MAX_NODES_EXACT,
                         betweenness_sample_k=BETWEENNESS_SAMPLE_K,
                         paths_sample_nodes=PATHS_SAMPLE_NODES):
    metrics = {"Graph": name}

    # Basic properties
    n_nodes = G.number_of_nodes()
    n_edges = G.number_of_edges()
    metrics["number_of_nodes"] = n_nodes
    metrics["number_of_edges"] = n_edges
    metrics["density"] = nx.density(G) if n_nodes > 1 else 0
    metrics["connected"] = nx.is_connected(G) if n_nodes > 0 else False
    metrics["bipartite"] = nx.is_bipartite(G) if n_nodes > 0 else False

    try:
        metrics["planar"] = nx.check_planarity(G)[0]
    except Exception:
        metrics["planar"] = False

    # Components
    components = list(nx.connected_components(G))
    metrics["number_connected_components"] = len(components)
    largest_cc = max(components, key=len) if components else set()
    metrics["size_largest_component"] = len(largest_cc) if largest_cc else 0

    # Prepare largest connected component for path-based metrics
    if metrics["connected"]:
        H_lcc = H
    else:
        H_lcc = H.subgraph(largest_cc).copy() if largest_cc else nx.Graph()

    n_lcc = H_lcc.number_of_nodes()
    use_exact_paths = (n_lcc <= max_nodes_exact) and (n_lcc > 0)

    # Path-based metrics
    if n_lcc == 0:
        metrics["diameter"] = -1
        metrics["radius"] = -1
        metrics["avg_shortest_path_length"] = -1
        metrics["wiener_index"] = -1
    else:
        if use_exact_paths:
            try:
                metrics["diameter"] = nx.diameter(H_lcc, weight='distance')
                metrics["radius"] = nx.radius(H_lcc, weight='distance')
                metrics["avg_shortest_path_length"] = nx.average_shortest_path_length(H_lcc, weight='distance')
                # Wiener index calculation
                wiener = 0
                paths = dict(nx.all_pairs_dijkstra_path_length(H_lcc, weight='distance'))
                nodes_list = list(H_lcc.nodes())
                for i in range(len(nodes_list)):
                    for j in range(i+1, len(nodes_list)):
                        u = nodes_list[i]
                        v = nodes_list[j]
                        wiener += paths[u][v]
                metrics["wiener_index"] = wiener
            except Exception:
                metrics["diameter"] = -1
                metrics["radius"] = -1
                metrics["avg_shortest_path_length"] = -1
                metrics["wiener_index"] = -1
        else:
            sample_nodes = sample_nodes_list(H_lcc, min(paths_sample_nodes, n_lcc))
            ecc = approx_eccentricity(H_lcc, sample_nodes)
            if ecc:
                metrics["diameter"] = max(ecc.values()) if ecc else -1
                metrics["radius"] = min(ecc.values()) if ecc else -1
            else:
                metrics["diameter"] = -1
                metrics["radius"] = -1
            metrics["avg_shortest_path_length"] = approx_average_shortest_path(H_lcc, sample_nodes)
            metrics["wiener_index"] = approx_wiener_index(H_lcc, sample_nodes)

    # Girth
    try:
        cycles = nx.cycle_basis(G)
        if cycles:
            metrics["girth"] = min(len(cycle) for cycle in cycles)
        else:
            metrics["girth"] = -1
    except Exception:
        metrics["girth"] = -1

    # Centrality metrics
    try:
        deg_cent = nx.degree_centrality(G)
        metrics["avg_degree_centrality"] = np.mean(list(deg_cent.values())) if deg_cent else -1
    except Exception:
        metrics["avg_degree_centrality"] = -1

    try:
        if n_nodes <= max_nodes_exact:
            betweenness = nx.betweenness_centrality(G)
        else:
            betweenness = nx.betweenness_centrality(G, k=betweenness_sample_k)
        metrics["avg_betweenness_centrality"] = np.mean(list(betweenness.values())) if betweenness else -1
    except Exception:
        metrics["avg_betweenness_centrality"] = -1

    try:
        if n_nodes <= max_nodes_exact:
            closeness = nx.closeness_centrality(G)
            metrics["avg_closeness_centrality"] = np.mean(list(closeness.values())) if closeness else -1
        else:
            metrics["avg_closeness_centrality"] = -1
    except Exception:
        metrics["avg_closeness_centrality"] = -1

    # Clustering and transitivity
    try:
        metrics["transitivity"] = nx.transitivity(G)
    except Exception:
        metrics["transitivity"] = -1

    try:
        clustering = nx.clustering(G)
        metrics["avg_clustering"] = np.mean(list(clustering.values())) if clustering else -1
    except Exception:
        metrics["avg_clustering"] = -1

    # Degeneracy
    try:
        if n_nodes <= max_nodes_exact:
            core_num = nx.core_number(G)
            metrics["degeneracy"] = max(core_num.values()) if core_num else -1
        else:
            metrics["degeneracy"] = -1
    except Exception:
        metrics["degeneracy"] = -1

    # Clique number
    try:
        if n_nodes <= max_nodes_exact:
            metrics["clique_number"] = nx.graph_clique_number(G)
        else:
            metrics["clique_number"] = -1
    except Exception:
        metrics["clique_number"] = -1

    # Spectral properties
    try:
        if n_nodes <= max_nodes_exact and n_nodes > 0:
            A = nx.to_numpy_array(G)
            eigenvalues = np.linalg.eigvals(A).real
            metrics["graph_energy"] = np.sum(np.abs(eigenvalues))
            sorted_eigen = np.sort(eigenvalues)
            metrics["top_eigenvalue"] = sorted_eigen[-1] if len(eigenvalues) > 0 else -1
            metrics["second_eigenvalue"] = sorted_eigen[-2] if len(eigenvalues) > 1 else -1
        else:
            metrics["graph_energy"] = -1
            metrics["top_eigenvalue"] = -1
            metrics["second_eigenvalue"] = -1
    except Exception:
        metrics["graph_energy"] = -1
        metrics["top_eigenvalue"] = -1
        metrics["second_eigenvalue"] = -1

    # Max flow
    try:
        if n_nodes >= 2:
            nodes_list = list(G.nodes())
            flow_value = nx.maximum_flow_value(G, nodes_list[0], nodes_list[1], capacity='weight')
            metrics["max_flow"] = flow_value
        else:
            metrics["max_flow"] = -1
    except Exception:
        metrics["max_flow"] = -1

    # Matching number
    try:
        if n_nodes > 0:
            matching = nx.max_weight_matching(G)
            metrics["matching_number"] = len(matching)  # Number of edges in matching
        else:
            metrics["matching_number"] = -1
    except Exception:
        metrics["matching_number"] = -1

    # Algebraic connectivity
    try:
        if n_nodes > 0 and nx.is_connected(G):
            metrics["algebraic_connectivity"] = nx.algebraic_connectivity(G)
        else:
            metrics["algebraic_connectivity"] = -1
    except Exception:
        metrics["algebraic_connectivity"] = -1

    return metrics

# ---------------------------
# SAMPLE PARAGRAPHS       মৌরি মরিয়ম
# ---------------------------
paragraphs = [
   """

   """,
      """

   """
]

# Storage for results
graph_features_list = []

# ---------------------------
# MAIN PROCESSING LOOP
# ---------------------------
t_start_all = time.time()
for idx, paragraph in enumerate(paragraphs, 1):
    t0 = time.time()
    print(f"\n--- Processing Paragraph {idx} ---")

    # Preprocess the text
    tokens = preprocess_text(paragraph)
    # print(f"Tokens: ",tokens)

    if not tokens:
        print(f"Paragraph {idx} is empty after preprocessing — skipping.")
        continue

    print(f"Tokens: {len(tokens)} — creating PMI co-occurrence graph...")

    # Create PMI-based graph
    G = create_pmi_graph(tokens)

    print(f"Graph created: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

    # Create distance-weighted version for path calculations
    H = G.copy()
    for u, v, data in H.edges(data=True):
        wt = data.get("weight", 1e-6)
        # Convert PMI weight to distance (inverse relationship)
        data["distance"] = 1.0 / (wt if wt > 1e-6 else 1e-6)

    # Compute comprehensive graph metrics
    graph_data = compute_graph_metrics(G, H, idx)
    graph_features_list.append(graph_data)

    t1 = time.time()
    print(f"Paragraph {idx} completed in {t1 - t0:.1f}s.")

# ---------------------------
# CREATE FINAL TABLE & SAVE
# ---------------------------
if graph_features_list:
    df_graphs = pd.DataFrame(graph_features_list)

    # Calculate average row
    avg_row = df_graphs.mean(numeric_only=True)
    avg_row["Graph"] = "Average"
    df_graphs = pd.concat([df_graphs, pd.DataFrame([avg_row])], ignore_index=True)

    # Define column order
    cols = [
        "Graph", "number_of_nodes", "number_of_edges", "density", "connected",
        "bipartite", "planar", "number_connected_components", "size_largest_component",
        "diameter", "radius", "avg_shortest_path_length", "wiener_index", "girth",
        "avg_degree_centrality", "avg_betweenness_centrality", "avg_closeness_centrality",
        "transitivity", "avg_clustering", "degeneracy", "clique_number", "graph_energy",
        "top_eigenvalue", "second_eigenvalue", "max_flow", "matching_number", "algebraic_connectivity"
    ]

    # Reorder columns
    df_graphs = df_graphs[cols]

    # Save to Excel
    print("\nWriting results to Excel...")
    df_graphs.to_excel(OUTPUT_EXCEL, index=False)

    print(f"✅ PMI co-occurrence graph analysis saved to {OUTPUT_EXCEL}")
    print(f"📊 Analyzed {len(graph_features_list)} paragraphs")
    print(f"⏱️ Total processing time: {time.time() - t_start_all:.1f}s")

    # Display summary
    print("\n--- Summary Statistics ---")
    print(f"Average nodes per graph: {df_graphs['number_of_nodes'].iloc[:-1].mean():.2f}")
    print(f"Average edges per graph: {df_graphs['number_of_edges'].iloc[:-1].mean():.2f}")
    print(f"Average density: {df_graphs['density'].iloc[:-1].mean():.4f}")
    print(f"Connected graphs: {df_graphs['connected'].iloc[:-1].sum()}/{len(graph_features_list)}")

else:
    print("No valid paragraphs were processed. Please check your input text.")

print("\n🎉 PMI Co-occurrence Graph Analysis Complete!")

PMI Co-occurrence Graph Analysis System Initialized


--- Processing Paragraph 1 ---
Tokens: 308 — creating PMI co-occurrence graph...
Graph created: 232 nodes, 894 edges
Paragraph 1 completed in 1.4s.

--- Processing Paragraph 2 ---
Tokens: 318 — creating PMI co-occurrence graph...
Graph created: 250 nodes, 926 edges
Paragraph 2 completed in 2.4s.

--- Processing Paragraph 3 ---
Tokens: 320 — creating PMI co-occurrence graph...
Graph created: 240 nodes, 908 edges
Paragraph 3 completed in 1.5s.

--- Processing Paragraph 4 ---
Tokens: 278 — creating PMI co-occurrence graph...
Graph created: 211 nodes, 796 edges
Paragraph 4 completed in 1.0s.

--- Processing Paragraph 5 ---
Tokens: 310 — creating PMI co-occurrence graph...
Graph created: 221 nodes, 866 edges
Paragraph 5 completed in 1.2s.

--- Processing Paragraph 6 ---
Tokens: 265 — creating PMI co-occurrence graph...
Graph created: 196 nodes, 742 edges
Paragraph 6 completed in 0.9s.

--- Processing Paragraph 7 ---
Tokens: 308 — creatin

##English

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import re
import os
import random
import time
import warnings
from collections import defaultdict, Counter
import itertools
import math

# Optional imports for advanced tokenization
try:
    import nltk
    from nltk.tokenize import word_tokenize
    from nltk.stem import WordNetLemmatizer
    # Download required NLTK data (run once)
    nltk.download('punkt')
    nltk.download('punkt_tab')
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    NLTK_AVAILABLE = True
except ImportError:
    NLTK_AVAILABLE = False
    print("Warning: NLTK not available. Using simple tokenization only.")

try:
    import spacy
    # Load spaCy model: python -m spacy download en_core_web_sm
    try:
        nlp = spacy.load("en_core_web_sm")
        SPACY_AVAILABLE = True
    except:
        SPACY_AVAILABLE = False
        print("Warning: spaCy model not loaded. Using NLTK or simple tokenization.")
except ImportError:
    SPACY_AVAILABLE = False

warnings.filterwarnings("ignore")

# ---------------------------
# CONFIG
# ---------------------------
# Input/Output Configuration
INPUT_FILE = "AI English.txt"  # Your input text file
OUTPUT_DIR = "english_outputs_PMI"
FILE_NAME = "AI_english_pmi_graph_analysis_adv_nltk_nolemma.xlsx"
OUTPUT_EXCEL = os.path.join(OUTPUT_DIR, FILE_NAME)

# PREPROCESSING OPTIONS - CHANGE THESE TO TEST DIFFERENT COMBINATIONS
USE_ADVANCED_TOKENIZATION = True  # True = NLTK/spaCy, False = simple whitespace
TOKENIZATION_METHOD = "nltk"       # "nltk" or "spacy" (only if USE_ADVANCED_TOKENIZATION=True)
USE_LEMMATIZATION = False       # True = lemmatize tokens, False = keep original

# Thresholds for switching to approximate computations
MAX_NODES_EXACT = 500           # <= this: do exact heavy computations
BETWEENNESS_SAMPLE_K = 100      # 'k' parameter for approx betweenness
PATHS_SAMPLE_NODES = 100        # number of source nodes to sample for path-based metrics
RANDOM_SEED = 42

# PMI Parameters
WINDOW_SIZE = 4                 # sliding window size for co-occurrence
MIN_PMI = 0.0                   # minimum PMI threshold for edge creation

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print("=" * 70)
print("English PMI Co-occurrence Graph Analysis System")
print("=" * 70)
print(f"Configuration:")
print(f"  - Advanced Tokenization: {USE_ADVANCED_TOKENIZATION}")
if USE_ADVANCED_TOKENIZATION:
    print(f"  - Method: {TOKENIZATION_METHOD.upper()}")
print(f"  - Lemmatization: {USE_LEMMATIZATION}")
print(f"  - Window Size: {WINDOW_SIZE}")
print(f"  - Min PMI: {MIN_PMI}")
print("=" * 70 + "\n")

# ---------------------------
# FILE READING FUNCTION
# ---------------------------
def read_paragraphs_from_file(filepath):
    """
    Read paragraphs from file with format:
    \"\"\"
    paragraph text here
    \"\"\",
    \"\"\"
    another paragraph
    \"\"\"
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()

        # Split by triple quotes
        parts = content.split('"""')

        # Extract non-empty paragraphs (odd indices contain the text)
        paragraphs = []
        for i in range(1, len(parts), 2):
            text = parts[i].strip()
            # Remove trailing comma if present
            text = text.rstrip(',').strip()
            if text:
                paragraphs.append(text)

        print(f"✅ Loaded {len(paragraphs)} paragraphs from {filepath}\n")
        return paragraphs

    except FileNotFoundError:
        print(f"❌ Error: File '{filepath}' not found!")
        print(f"Please create the file or update INPUT_FILE path.\n")
        return []
    except Exception as e:
        print(f"❌ Error reading file: {e}\n")
        return []

# ---------------------------
# TEXT PREPROCESSING FUNCTIONS
# ---------------------------

# def clean_english_text(text):
#     """Remove punctuation, keep only English letters and spaces, convert to lowercase"""
#     # Convert to lowercase
#     text = text.lower()
#     # Remove punctuation but keep letters, numbers, and spaces
#     text = re.sub(r'[^a-z0-9\s]', ' ', text)
#     # Remove extra whitespace
#     text = re.sub(r'\s+', ' ', text)
#     return text.strip()

def clean_english_text(text):
    """Remove punctuation, corrupted Unicode, keep only English letters and spaces, convert to lowercase"""
    # Convert to lowercase
    text = text.lower()

    # Remove common corrupted Unicode patterns
    # These often appear when UTF-8 is incorrectly decoded as Latin-1
    corrupted_patterns = [
        'â€™', 'â€œ', 'â€', 'â€"', 'â€"', 'â€¢',
        'â€¦', 'â€˜', 'â€', 'â€ ', 'â€¡', 'â€°',
        'â', '€', '™', 'œ', '"', '—', '•', '¦', '˜'
    ]
    for pattern in corrupted_patterns:
        text = text.replace(pattern, ' ')

    # Remove any remaining non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)

    # Keep only English letters and spaces (remove numbers if not needed)
    text = re.sub(r'[^a-z\s]', ' ', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)

    return text.strip()


def simple_tokenize(text):
    """
    Simple tokenization by splitting on whitespace
    PROS: Fast, simple, no dependencies
    CONS: Doesn't handle contractions well
    """
    return text.split()

def advanced_tokenize(text, method="nltk"):
    """
    Advanced tokenization using NLTK or spaCy
    PROS: Better handling of contractions, punctuation
    CONS: Requires external libraries, slower
    """
    if method == "nltk" and NLTK_AVAILABLE:
        return word_tokenize(text)
    elif method == "spacy" and SPACY_AVAILABLE:
        doc = nlp(text)
        return [token.text for token in doc]
    else:
        print(f"Warning: {method} not available, falling back to simple tokenization")
        return simple_tokenize(text)

def lemmatize_tokens(tokens, use_lemmatization=True):
    """
    Optional lemmatization to reduce words to their base form
    Example: running -> run, better -> good
    PROS: Groups word variants, reduces vocabulary
    CONS: May lose some semantic nuance, slower
    """
    if not use_lemmatization:
        return tokens

    if NLTK_AVAILABLE:
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.lemmatize(token) for token in tokens]
    elif SPACY_AVAILABLE:
        doc = nlp(" ".join(tokens))
        return [token.lemma_ for token in doc]
    else:
        print("Warning: Lemmatization requested but no library available")
        return tokens

def remove_stopwords(tokens):
    """
    Remove common English stopwords
    These are high-frequency words that don't carry much semantic meaning
    """
    stopwords = {
        'the', 'be', 'to', 'of', 'and', 'a', 'in', 'that', 'have', 'i',
        'it', 'for', 'not', 'on', 'with', 'he', 'as', 'you', 'do', 'at',
        'this', 'but', 'his', 'by', 'from', 'they', 'we', 'say', 'her', 'she',
        'or', 'an', 'will', 'my', 'one', 'all', 'would', 'there', 'their',
        'what', 'so', 'up', 'out', 'if', 'about', 'who', 'get', 'which', 'go',
        'me', 'when', 'make', 'can', 'like', 'time', 'no', 'just', 'him', 'know',
        'take', 'people', 'into', 'year', 'your', 'good', 'some', 'could', 'them',
        'see', 'other', 'than', 'then', 'now', 'look', 'only', 'come', 'its', 'over',
        'think', 'also', 'back', 'after', 'use', 'two', 'how', 'our', 'work',
        'first', 'well', 'way', 'even', 'new', 'want', 'because', 'any', 'these',
        'give', 'day', 'most', 'us', 'is', 'was', 'are', 'been', 'has', 'had',
        'were', 'said', 'did', 'having', 'may', 'should', 'am', 'being', 'does'
    }
    # Also filter very short tokens (1 character)
    return [word for word in tokens if word not in stopwords and len(word) > 1]

def preprocess_text(text):
    """
    Complete preprocessing pipeline with configurable options

    Pipeline:
    1. Clean text (lowercase, remove punctuation)
    2. Tokenize (simple or advanced)
    3. Lemmatize (optional)
    4. Remove stopwords
    """
    # Step 1: Clean
    cleaned = clean_english_text(text)

    # Step 2: Tokenize
    if USE_ADVANCED_TOKENIZATION:
        tokens = advanced_tokenize(cleaned, method=TOKENIZATION_METHOD)
    else:
        tokens = simple_tokenize(cleaned)

    # Step 3: Optional lemmatization
    tokens = lemmatize_tokens(tokens, use_lemmatization=USE_LEMMATIZATION)

    # Step 4: Remove stopwords
    tokens = remove_stopwords(tokens)

    return tokens

# ---------------------------
# PMI CO-OCCURRENCE FUNCTIONS
# ---------------------------
def calculate_pmi_edges(tokens, window_size=WINDOW_SIZE, min_pmi=MIN_PMI):
    """Calculate PMI-based edges from tokens"""
    if len(tokens) < 2:
        return {}

    word_freq = Counter()
    pair_freq = Counter()
    total_windows = 0

    # Count word frequencies and co-occurrences within sliding windows
    for i in range(len(tokens) - window_size + 1):
        window = tokens[i:i + window_size]
        total_windows += 1

        # Count individual words
        for word in window:
            word_freq[word] += 1

        # Count word pairs
        for w1, w2 in itertools.combinations(window, 2):
            pair = tuple(sorted((w1, w2)))
            pair_freq[pair] += 1

    # Calculate PMI scores and create edges
    edges = {}
    for (w1, w2), c in pair_freq.items():
        if total_windows == 0:
            continue

        p_xy = c / total_windows
        p_x = word_freq[w1] / (total_windows * window_size)
        p_y = word_freq[w2] / (total_windows * window_size)

        if p_x > 0 and p_y > 0:
            pmi = math.log(p_xy / (p_x * p_y))
            if pmi > min_pmi:
                edges[(w1, w2)] = pmi

    return edges

def create_pmi_graph(tokens):
    """Create NetworkX graph from tokens using PMI co-occurrence"""
    edges = calculate_pmi_edges(tokens)

    G = nx.Graph()

    # Add all tokens as nodes
    for token in set(tokens):
        G.add_node(token)

    # Add edges based on PMI scores
    for (w1, w2), pmi_score in edges.items():
        G.add_edge(w1, w2, weight=pmi_score)

    return G

# ---------------------------
# UTILITIES FOR APPROXIMATIONS
# ---------------------------
def sample_nodes_list(G, k):
    n = G.number_of_nodes()
    nodes = list(G.nodes())
    if n <= k:
        return nodes
    return random.sample(nodes, k)

def approx_average_shortest_path(H, sample_nodes):
    lengths_accum = []
    count = 0
    for s in sample_nodes:
        try:
            lens = nx.single_source_dijkstra_path_length(H, s, weight="distance")
            lengths_accum.extend(list(lens.values()))
            count += len(lens)
        except Exception:
            continue
    if count == 0:
        return -1
    return float(sum(lengths_accum) / count)

def approx_wiener_index(H, sample_nodes):
    wsum = 0.0
    pairs_count = 0
    for s in sample_nodes:
        try:
            lens = nx.single_source_dijkstra_path_length(H, s, weight="distance")
        except Exception:
            continue
        for t, d in lens.items():
            if s < t:
                wsum += d
                pairs_count += 1
    return float(wsum) if pairs_count > 0 else -1

def approx_eccentricity(H, sample_nodes):
    ecc = {}
    for s in sample_nodes:
        try:
            lens = nx.single_source_dijkstra_path_length(H, s, weight="distance")
            if len(lens) == 0:
                ecc[s] = -1
            else:
                ecc[s] = max(lens.values())
        except Exception:
            ecc[s] = -1
    return ecc

# ---------------------------
# COMPREHENSIVE GRAPH METRICS
# ---------------------------
def compute_graph_metrics(G, H, name, max_nodes_exact=MAX_NODES_EXACT,
                         betweenness_sample_k=BETWEENNESS_SAMPLE_K,
                         paths_sample_nodes=PATHS_SAMPLE_NODES):
    metrics = {"Graph": name}

    # Basic properties
    n_nodes = G.number_of_nodes()
    n_edges = G.number_of_edges()
    metrics["number_of_nodes"] = n_nodes
    metrics["number_of_edges"] = n_edges
    metrics["density"] = nx.density(G) if n_nodes > 1 else 0
    metrics["connected"] = nx.is_connected(G) if n_nodes > 0 else False
    metrics["bipartite"] = nx.is_bipartite(G) if n_nodes > 0 else False

    try:
        metrics["planar"] = nx.check_planarity(G)[0]
    except Exception:
        metrics["planar"] = False

    # Components
    components = list(nx.connected_components(G))
    metrics["number_connected_components"] = len(components)
    largest_cc = max(components, key=len) if components else set()
    metrics["size_largest_component"] = len(largest_cc) if largest_cc else 0

    # Prepare largest connected component for path-based metrics
    if metrics["connected"]:
        H_lcc = H
    else:
        H_lcc = H.subgraph(largest_cc).copy() if largest_cc else nx.Graph()

    n_lcc = H_lcc.number_of_nodes()
    use_exact_paths = (n_lcc <= max_nodes_exact) and (n_lcc > 0)

    # Path-based metrics
    if n_lcc == 0:
        metrics["diameter"] = -1
        metrics["radius"] = -1
        metrics["avg_shortest_path_length"] = -1
        metrics["wiener_index"] = -1
    else:
        if use_exact_paths:
            try:
                metrics["diameter"] = nx.diameter(H_lcc, weight='distance')
                metrics["radius"] = nx.radius(H_lcc, weight='distance')
                metrics["avg_shortest_path_length"] = nx.average_shortest_path_length(H_lcc, weight='distance')
                # Wiener index calculation
                wiener = 0
                paths = dict(nx.all_pairs_dijkstra_path_length(H_lcc, weight='distance'))
                nodes_list = list(H_lcc.nodes())
                for i in range(len(nodes_list)):
                    for j in range(i+1, len(nodes_list)):
                        u = nodes_list[i]
                        v = nodes_list[j]
                        wiener += paths[u][v]
                metrics["wiener_index"] = wiener
            except Exception:
                metrics["diameter"] = -1
                metrics["radius"] = -1
                metrics["avg_shortest_path_length"] = -1
                metrics["wiener_index"] = -1
        else:
            sample_nodes = sample_nodes_list(H_lcc, min(paths_sample_nodes, n_lcc))
            ecc = approx_eccentricity(H_lcc, sample_nodes)
            if ecc:
                metrics["diameter"] = max(ecc.values()) if ecc else -1
                metrics["radius"] = min(ecc.values()) if ecc else -1
            else:
                metrics["diameter"] = -1
                metrics["radius"] = -1
            metrics["avg_shortest_path_length"] = approx_average_shortest_path(H_lcc, sample_nodes)
            metrics["wiener_index"] = approx_wiener_index(H_lcc, sample_nodes)

    # Girth
    try:
        cycles = nx.cycle_basis(G)
        if cycles:
            metrics["girth"] = min(len(cycle) for cycle in cycles)
        else:
            metrics["girth"] = -1
    except Exception:
        metrics["girth"] = -1

    # Centrality metrics
    try:
        deg_cent = nx.degree_centrality(G)
        metrics["avg_degree_centrality"] = np.mean(list(deg_cent.values())) if deg_cent else -1
    except Exception:
        metrics["avg_degree_centrality"] = -1

    try:
        if n_nodes <= max_nodes_exact:
            betweenness = nx.betweenness_centrality(G)
        else:
            betweenness = nx.betweenness_centrality(G, k=betweenness_sample_k)
        metrics["avg_betweenness_centrality"] = np.mean(list(betweenness.values())) if betweenness else -1
    except Exception:
        metrics["avg_betweenness_centrality"] = -1

    try:
        if n_nodes <= max_nodes_exact:
            closeness = nx.closeness_centrality(G)
            metrics["avg_closeness_centrality"] = np.mean(list(closeness.values())) if closeness else -1
        else:
            metrics["avg_closeness_centrality"] = -1
    except Exception:
        metrics["avg_closeness_centrality"] = -1

    # Clustering and transitivity
    try:
        metrics["transitivity"] = nx.transitivity(G)
    except Exception:
        metrics["transitivity"] = -1

    try:
        clustering = nx.clustering(G)
        metrics["avg_clustering"] = np.mean(list(clustering.values())) if clustering else -1
    except Exception:
        metrics["avg_clustering"] = -1

    # Degeneracy
    try:
        if n_nodes <= max_nodes_exact:
            core_num = nx.core_number(G)
            metrics["degeneracy"] = max(core_num.values()) if core_num else -1
        else:
            metrics["degeneracy"] = -1
    except Exception:
        metrics["degeneracy"] = -1

    # Clique number
    try:
        if n_nodes <= max_nodes_exact:
            metrics["clique_number"] = nx.graph_clique_number(G)
        else:
            metrics["clique_number"] = -1
    except Exception:
        metrics["clique_number"] = -1

    # Spectral properties
    try:
        if n_nodes <= max_nodes_exact and n_nodes > 0:
            A = nx.to_numpy_array(G)
            eigenvalues = np.linalg.eigvals(A).real
            metrics["graph_energy"] = np.sum(np.abs(eigenvalues))
            sorted_eigen = np.sort(eigenvalues)
            metrics["top_eigenvalue"] = sorted_eigen[-1] if len(eigenvalues) > 0 else -1
            metrics["second_eigenvalue"] = sorted_eigen[-2] if len(eigenvalues) > 1 else -1
        else:
            metrics["graph_energy"] = -1
            metrics["top_eigenvalue"] = -1
            metrics["second_eigenvalue"] = -1
    except Exception:
        metrics["graph_energy"] = -1
        metrics["top_eigenvalue"] = -1
        metrics["second_eigenvalue"] = -1

    # Max flow
    try:
        if n_nodes >= 2:
            nodes_list = list(G.nodes())
            flow_value = nx.maximum_flow_value(G, nodes_list[0], nodes_list[1], capacity='weight')
            metrics["max_flow"] = flow_value
        else:
            metrics["max_flow"] = -1
    except Exception:
        metrics["max_flow"] = -1

    # Matching number
    try:
        if n_nodes > 0:
            matching = nx.max_weight_matching(G)
            metrics["matching_number"] = len(matching)
        else:
            metrics["matching_number"] = -1
    except Exception:
        metrics["matching_number"] = -1

    # Algebraic connectivity
    try:
        if n_nodes > 0 and nx.is_connected(G):
            metrics["algebraic_connectivity"] = nx.algebraic_connectivity(G)
        else:
            metrics["algebraic_connectivity"] = -1
    except Exception:
        metrics["algebraic_connectivity"] = -1

    return metrics

# ---------------------------
# MAIN PROCESSING LOOP
# ---------------------------
def main():
    # Read paragraphs from file
    paragraphs = read_paragraphs_from_file(INPUT_FILE)

    if not paragraphs:
        print("❌ No paragraphs to process. Exiting.")
        return

    # Storage for results
    graph_features_list = []

    t_start_all = time.time()

    for idx, paragraph in enumerate(paragraphs, 1):
        t0 = time.time()
        print(f"\n{'─' * 70}")
        print(f"Processing Paragraph {idx}/{len(paragraphs)}")
        print(f"{'─' * 70}")

        # Preprocess the text
        tokens = preprocess_text(paragraph)

        if not tokens:
            print(f"⚠️  Paragraph {idx} is empty after preprocessing — skipping.")
            continue

        print(f"📝 Tokens extracted: {len(tokens)}")
        print(f"🔤 Sample tokens: {tokens[:10]}..." if len(tokens) > 10 else f"🔤 Tokens: {tokens}")

        # Create PMI-based graph
        print(f"🔨 Building PMI co-occurrence graph...")
        G = create_pmi_graph(tokens)

        print(f"✅ Graph created: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

        # Create distance-weighted version for path calculations
        H = G.copy()
        for u, v, data in H.edges(data=True):
            wt = data.get("weight", 1e-6)
            data["distance"] = 1.0 / (wt if wt > 1e-6 else 1e-6)

        # Compute comprehensive graph metrics
        print(f"📊 Computing graph metrics...")
        graph_data = compute_graph_metrics(G, H, f"Para_{idx}")
        graph_features_list.append(graph_data)

        t1 = time.time()
        print(f"⏱️  Completed in {t1 - t0:.2f}s")

    # ---------------------------
    # CREATE FINAL TABLE & SAVE
    # ---------------------------
    if graph_features_list:
        print(f"\n{'=' * 70}")
        print("Creating Excel output...")
        print(f"{'=' * 70}")

        df_graphs = pd.DataFrame(graph_features_list)

        # Calculate average row
        avg_row = df_graphs.mean(numeric_only=True)
        avg_row["Graph"] = "Average"
        df_graphs = pd.concat([df_graphs, pd.DataFrame([avg_row])], ignore_index=True)

        # Define column order
        cols = [
            "Graph", "number_of_nodes", "number_of_edges", "density", "connected",
            "bipartite", "planar", "number_connected_components", "size_largest_component",
            "diameter", "radius", "avg_shortest_path_length", "wiener_index", "girth",
            "avg_degree_centrality", "avg_betweenness_centrality", "avg_closeness_centrality",
            "transitivity", "avg_clustering", "degeneracy", "clique_number", "graph_energy",
            "top_eigenvalue", "second_eigenvalue", "max_flow", "matching_number", "algebraic_connectivity"
        ]

        # Reorder columns
        df_graphs = df_graphs[cols]

        # Save to Excel
        df_graphs.to_excel(OUTPUT_EXCEL, index=False)

        print(f"\n✅ Results saved to: {OUTPUT_EXCEL}")
        print(f"📊 Total paragraphs analyzed: {len(graph_features_list)}")
        print(f"⏱️  Total processing time: {time.time() - t_start_all:.2f}s")

        # Display summary statistics
        print(f"\n{'=' * 70}")
        print("SUMMARY STATISTICS")
        print(f"{'=' * 70}")
        print(f"Average nodes per graph:     {df_graphs['number_of_nodes'].iloc[:-1].mean():.2f}")
        print(f"Average edges per graph:     {df_graphs['number_of_edges'].iloc[:-1].mean():.2f}")
        print(f"Average density:             {df_graphs['density'].iloc[:-1].mean():.4f}")
        print(f"Connected graphs:            {df_graphs['connected'].iloc[:-1].sum()}/{len(graph_features_list)}")
        print(f"Average clustering:          {df_graphs['avg_clustering'].iloc[:-1].mean():.4f}")
        print(f"{'=' * 70}")

    else:
        print("\n❌ No valid paragraphs were processed. Please check your input file.")

    print("\n🎉 PMI Co-occurrence Graph Analysis Complete!\n")

# ---------------------------
# RUN THE ANALYSIS
# ---------------------------
if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


English PMI Co-occurrence Graph Analysis System
Configuration:
  - Advanced Tokenization: True
  - Method: NLTK
  - Lemmatization: False
  - Window Size: 4
  - Min PMI: 0.0

✅ Loaded 300 paragraphs from AI English.txt


──────────────────────────────────────────────────────────────────────
Processing Paragraph 1/300
──────────────────────────────────────────────────────────────────────
📝 Tokens extracted: 170
🔤 Sample tokens: ['little', 'girl', 'named', 'lily', 'found', 'needle', 'room', 'knew', 'difficult', 'play']...
🔨 Building PMI co-occurrence graph...
✅ Graph created: 91 nodes, 410 edges
📊 Computing graph metrics...
⏱️  Completed in 0.24s

──────────────────────────────────────────────────────────────────────
Processing Paragraph 2/300
──────────────────────────────────────────────────────────────────────
📝 Tokens extracted: 162
🔤 Sample tokens: ['once', 'upon', 'land', 'full', 'trees', 'little', 'cherry', 'tree', 'cherry', 'tree']...
🔨 Building PMI co-occurrence graph...
✅ Graph 